In [4]:
print("Hello, world")

Hello, world


In [6]:
!pip install numpy usearch

In [6]:
import sys  
sys.path.insert(1, '../src')
import recsys_support.support_func as rs

In [1]:
import numpy as np
import pandas as pd
from usearch.index import Index
from sklearn.preprocessing import RobustScaler

In [2]:
index = Index(ndim=3)

vector = np.array([0.2, 0.6, 0.4])
index.add(42, vector)

matches = index.search(vector, 10)
assert matches[0].key == 42
assert matches[0].distance <= 0.001
#assert np.allclose(index[42], vector)

### Load Data

In [3]:
df_base = pd.read_parquet("../tmp/cleared_df_base.parquet")

In [4]:
df_train = pd.read_parquet("../tmp/cleared_df_train.parquet")

In [5]:
def checkRobustScaler(base, train):
    scaler = RobustScaler()
    scaler.fit(base)

    tbase = scaler.transform(base)
    tbase = pd.DataFrame(tbase, columns=base.columns)
    tbase["Id"] = base.index
    tbase = tbase.set_index(["Id"])
    
    ttrain = scaler.transform(train[base.columns])

    ttrain = pd.DataFrame(ttrain, columns=base.columns)
    ttrain["Id"] = train.index
    ttrain = ttrain.set_index(["Id"])
    ttrain["Target"] = train["Target"]

    return tbase, ttrain, scaler

In [6]:
df_base, df_train, scaler = checkRobustScaler(df_base, df_train)

In [7]:
df_base = df_base.reset_index()
df_train = df_train.reset_index()

In [8]:
df_base_ids = df_base.pop("Id")
df_train_ids = df_train.pop("Id")
df_train_target = df_train.pop("Target")

In [9]:
df_base.head(5)

,0,1,2,3,4,5,6,7,8,9,...,61,62,63,64,65,66,67,68,69,71
0,-0.853397,0.460538,-0.412746,1.049610,1.690894,-0.841724,-0.099159,0.273315,-0.350781,0.678173,...,-0.602186,0.886879,0.116154,0.041450,-0.515980,-1.696136,-0.389094,0.219288,-0.027103,-0.218730
1,1.528277,0.782926,-0.520104,-0.743363,-0.875959,-0.364060,0.713246,-0.655240,-0.491795,0.839484,...,-1.348559,-0.976363,0.181517,-0.072084,-0.633299,0.158452,0.012873,1.279020,0.029052,-0.195393
2,0.946431,-0.245443,0.292514,0.502795,0.630760,-0.706570,1.004428,0.482669,0.596146,0.478435,...,-0.288257,0.051959,0.321153,-0.994906,0.663859,-0.919329,0.593723,0.074829,0.020445,0.287116
3,-0.038115,-0.591736,-0.880887,-1.440728,-0.573874,0.332243,0.082551,-0.631718,-0.219211,0.149039,...,-1.406098,0.194882,-1.167681,0.126802,-0.301341,-0.658991,-0.608587,0.758547,0.052023,-0.434720
4,0.398938,-0.224341,1.752062,0.232106,-1.714228,-1.031150,0.051814,0.283002,-0.203272,-0.133156,...,0.423585,0.529178,0.623368,-0.100573,0.421821,-0.428615,-0.358680,-0.117109,0.682816,0.874164


In [10]:
df_train.head(5)

,0,1,2,3,4,5,6,7,8,9,...,61,62,63,64,65,66,67,68,69,71
0,0.956813,1.468961,0.034035,-1.398175,1.222708,-0.390937,0.158339,0.203191,-1.744734,-1.089301,...,-0.115111,-0.669636,1.006160,-0.034157,-0.677278,-0.781285,0.152772,-0.149669,0.672481,0.404513
1,-0.045694,-0.182307,0.235551,-1.146973,0.144349,-0.305307,0.693155,-1.152057,0.120621,0.196990,...,-0.498066,0.936504,-0.854166,0.042657,-0.090459,0.145681,0.023242,0.183252,0.522692,-0.047985
2,1.072261,-0.620782,-1.447945,-1.265179,1.264912,0.541482,0.950244,-0.594990,0.221400,1.031093,...,-0.636131,0.657624,1.231668,-0.283853,0.399591,-0.738588,-0.069405,1.674534,0.211982,0.465483
3,1.136416,0.154941,-1.433588,0.969449,-0.278345,-0.958763,-0.746723,0.677757,0.013062,1.394367,...,-0.024586,0.420605,0.278609,0.160191,0.468884,-1.506003,1.143543,0.100472,0.080862,0.512745
4,0.195197,0.947107,-0.302133,-0.010374,-0.884907,1.430006,0.213437,0.292315,0.241446,-0.235276,...,2.098284,0.241547,0.293211,-0.788086,-0.768877,-0.363233,0.263447,0.032787,-0.844928,0.490466


### Test Index

In [32]:
dims = df_base.shape[1]

index = Index(
    ndim=dims, # Define the number of dimensions in input vectors
    metric='inner_product', # Choose 'l2sq', 'haversine' or other metric, default = 'ip'
    dtype='f32', # Quantize to 'f16' or 'i8' if needed, default = 'f32'
    #connectivity=16, # Optional: Limit number of neighbors per graph node
    #expansion_add=128, # Optional: Control the recall of indexing
    #expansion_search=32, # Optional: Control the quality of the search
    multi=False, # Optional: Allow multiple vectors per key, default = False
)
#"cos": MetricKind.Cos,
#"cosine": MetricKind.Cos,
#ip": MetricKind.IP,
#"dot": MetricKind.IP,
#inner_product": MetricKind.IP,
#"l2sq": MetricKind.L2sq,
#"l2_sq": MetricKind.L2sq,
#"haversine": MetricKind.Haversine,
#"divergence": MetricKind.Divergence,
#"pearson": MetricKind.Pearson,
#"hamming": MetricKind.Hamming,
#"tanimoto": MetricKind.Tanimoto,
#"sorensen": MetricKind.Sorensen,


In [33]:
%%time
index.add(df_base.index, df_base.values)

CPU times: total: 47min 28s
Wall time: 2min 5s


array([      0,       1,       2, ..., 2918136, 2918137, 2918138],
      dtype=uint64)

In [43]:
%%time
matches = index.search(df_train.values, 10000)

CPU times: total: 2h 39min 36s
Wall time: 7min 16s


In [44]:
%%time
predict_ids = []
for i in matches:
    predict_ids.append([z.key for z in i])

CPU times: total: 8min 43s
Wall time: 8min 44s


In [45]:
# функция вычисления accuracy@n
def accuracy(targets, predict_ids, base_index):
    acc = 0
    for target, candidates in zip(targets.tolist(), predict_ids):
        acc += int(target in [base_index[number] for number in candidates])
    print(acc)
    return (acc / len(predict_ids) * 100)

In [46]:
%%time
accuracy(df_train_target, predict_ids, {k: df_base_ids.iloc[v] for k, v in enumerate(df_base_ids.index.to_list())})

556
CPU times: total: 15min 19s
Wall time: 15min 19s


0.5559999999999999